### Looking at a simple class

In [1]:
class Example:
    """A simple example class"""
    a = 3
    def f(self, p):
        """A function that takes a single argument"""
        return f'The parameters were {self} and {p}'

Which we can use in the expected manner

In [2]:
e = Example()
print(f'Class {Example}')
print(f'Instance {e}')
print(f'Value    {e.a}')
print(f'Method   {e.f}')
print(f'Calling the method with 3 {repr(e.f(3))}')

Class <class '__main__.Example'>
Instance <__main__.Example object at 0x105f28610>
Value    3
Method   <bound method Example.f of <__main__.Example object at 0x105f28610>>
Calling the method with 3 'The parameters were <__main__.Example object at 0x105f28610> and 3'


### Using values

We can update the `a` on the class and see it also change on the instance

In [3]:
Example.a = 4
print(Example.a)
print(e.a)

4
4


But if we update the `a` on the instance

In [4]:
print(e.a)
e.a += 1
print(e.a)

4
5


It doesn't change `a` on the class

In [5]:
print(Example.a)

4


And now changing it on the class won't touch the value on the instance

In [6]:
Example.a = 99
print(f'Class    "a" {Example.a}')
print(f'Instance "a" {e.a}')

Class    "a" 99
Instance "a" 5


If we add a new value to the class, the instance will get it

In [7]:
Example.b = 12
print(f'Class    "b" {Example.b}')
print(f'Instance "b" {e.b}')

Class    "b" 12
Instance "b" 12


But not the other way round

In [8]:
e.c = -1
print(f'Instance has "c" {hasattr(e, "c")}')
print(f'Instance "c" {e.c}')
print(f'Class has "c" {hasattr(Example, "c")}')
print(f'Class    "c" {Example.c}')

Instance has "c" True
Instance "c" -1
Class has "c" False


AttributeError: type object 'Example' has no attribute 'c'

### Using methods

We can call the method on the instance:

In [ ]:
e.f(4)

Or via the class, although then we need to pass in the instance (`self`) explicitly

In [ ]:
Example.f(e, 4)

and there's nothing special about `self`

In [ ]:
Example.f('not an instance', 4)

So we shouldn't be surprised that on the class it's a function, but on the instance it's a method, which gets passed the instance as its first argument

In [ ]:
print(f'Example.f is a {type(Example.f)}')
print(f'e.f is a {type(e.f)}')

**Aside** In Python, all methods are functions, and there's nothing special about the name `self`.

In [ ]:
def double(self, x):
    return x + x

Example.double = double
print(f'Class has "double" {hasattr(Example, "double")}')
print(f'Instance has "double" {hasattr(e, "double")}')
print(f'Example.double is a {type(Example.double)}')
print(f'e.double is a {type(e.double)}')
print(f'e.double(3) is {e.double(3)}')

### Can we define a class without `class`?

What type is our original class?

In [ ]:
print(type(Example))

Not, perhaps, what one might expect - I for one would have guessed it would be called "class".

Given the type of some other things

In [ ]:
print(type(1))
print(type('string'))

and the fact one can do:

In [ ]:
print(repr(int(1)))
print(repr(str(1)))

gives us a good guess at how we might create a class

```python
>>> help(type)
Help on class type in module builtins:

class type(object)
 |  type(object_or_name, bases, dict)
 |  type(object) -> the object's type
 |  type(name, bases, dict) -> a new type
```

And it works:

In [ ]:
EmptyClass = type('EmptyClass', (), {})

In [ ]:
print(type(EmptyClass))
print(repr(EmptyClass))

In [ ]:
ec = EmptyClass()
print(type(ec))
print(repr(ec))

In [ ]:
print(dir(EmptyClass))

In [ ]:
SimpleClass = type('SimpleClass', (), {'a': 1, 'f': Example.f})
sc = SimpleClass()
print(sc.a)
print(sc.f(1))

In [ ]:
def increment(self):
    return self + 1

new_int = type('NewInt', (int,), {'increment': increment})
x = new_int()
print(type(x))
print(repr(x))
print(x == 0)
print(x.increment)
print(x.increment())

In [ ]:
def stringify(x):
    return repr(x)

print(f'{stringify(3)!r}')

In [ ]:
e.stringify = stringify
print(f'{e.stringify(3)!r}')

In [ ]:
ByHand = type('ByHand', (), {'f': Example.f, 'a': 3})

In [ ]:
print(type(ByHand))

In [ ]:
bh = ByHand()
print(bh)

In [ ]:
print(bh.f('fred'))
print(bh.a)

In [ ]:
bh.b = 3
print(bh.b)

In [ ]:
def make_a_ByHand():
    return ByHand()

bh2 = make_a_ByHand()
print(bh2.a)

In [ ]:
print(dir(make_a_ByHand))

In [ ]:
def make_a_ByHand_class():
    def f(self, arg):
        return f'f called with {self!r} and {arg!r}'
    cls = type('ByHand', (), {})
    cls.a = 3
    cls.f = f
    return cls

In [ ]:
c = make_a_ByHand_class()
print(f'Class {c!r}')
print(f'Class value a {c.a!r}')
print(f'Class function {c.f(None, "fred")}')

In [ ]:
o = c()
print(f'Instance {o!r}')
print(f'Instance vaue a {o.a!r}')
print(f'Instance function {o.f("fred")}')

In [ ]:
make_a_ByHand_class.b = 3

In [ ]:
print(make_a_ByHand_class.b)

In [ ]:
type(make_a_ByHand_class)

In [ ]:
help(make_a_ByHand_class)

In [ ]:
make_a_ByHand_class.__doc__ = 'A function that returns a class it made'
help(make_a_ByHand_class)

In [ ]:
print(make_a_ByHand_class.__dict__)

### Can we construct an instance by hand?

Can we create an empty object and add things to it?

In [ ]:
o = object()

In [ ]:
o.a = 1

So that won't work

So we probably still have to use `type` to get to an empty mutable object

In [ ]:
eo_class = type('empty_object', (), {})
eo = eo_class()
print(type(eo))

And we know we can do

In [ ]:
eo.a = 1
print(eo.a)

In [ ]:
eo.a = eo.a + 1
print(eo.a)

Can we add a function *to the object* and have it be a method?

In [ ]:
def maybe_a_method(self, x):
    print(f'Maybe a method on {self} and {x}')
    
eo.f = maybe_a_method
print(eo.f(1))

In [ ]:
print(type(eo.f))

Normally, when we ask an instance for a method (`eo.f`), it gets looked up in the instance, isn't found there,
and is looked up in the class, which says
"I know what you're doing, that's a function you're looking up on me, so you must want a method back"

In [ ]:
class NoMethods: pass
def not_a_method(self): return 'Aha!'
NoMethods.f = not_a_method
nm = NoMethods()

print(type(f))
print(type(NoMethods.f))
print(type(nm.f))
print(nm.f())

But when our empty object is created from an empty class, with no methods.
Luckily there is a way:

In [ ]:
eo.f = not_a_method.__get__(eo, eo_class)
print(type(eo.f))
print(eo.f())

I don't propose to explain that (but am grateful to https://stackoverflow.com/a/46757134 for the example!).

If you want to learn more, then this is using the power of *descriptors*, which are at the heart of Python's attribute access

See the HOWTO at https://docs.python.org/3/howto/descriptor.html

There is also a more "understandable" way to do this.

Since we're wanting to "bind" the function to the object
(so that when we call it, the `self` argument is automatically put in),
we might be surprised to find out that there's a callable to do that:

In [ ]:
import types
eo.f = types.MethodType(not_a_method, eo)
print(type(eo.f))
print(eo.f())

In [ ]:
def pretend_instance(class_name, variable_dict, function_list):
    eo_class = type(class_name, (), variable_dict)
    eo = eo_class()
    for f in function_list:
        setattr(eo, f.__name__, types.MethodType(f, eo))
    return eo

In [ ]:
x = pretend_instance('ClassName', {'var': 3}, [not_a_method])
print(type(x))
print(x.var)
print(x.not_a_method())

In [ ]:
def bare_f(self, arg):
    return f'f called with {self!r} and {arg!r}'
print(type(bare_f))

In [ ]:
fn.f = bare_f
fn.a = 3
print(f'Calling f {fn.f(None, "fred")}')
print(f'Value a {fn.a}')

In [ ]:
fn.f(ByHand2, 'fred')

In [ ]:
make_a_ByHand_class.__call__()

In [ ]:
print(f'fn.__call__() {fn.__call__()!r}')
print(f'fn.f.__call__(1,2) {fn.f.__call__(1,2)!r}')

Any Python object with a `__call__` method may be called, and that is sufficient for an object to "act like" a function.

### Can we create a function without using `def`?

Well, there is lambda

In [ ]:
lamb = lambda x: x + 1

lamb(2)

although

1. That's another keyword
2. It's very limited in what it can do

> An anonymous inline function consisting of a single expression which is evaluated when the function is called

There is also `types.FunctionType`, which is similar in idea to our use of `type` to create classes.

(unfortunately, it's signature is implementation specific and may even change between Python versions)

```python
>>> help(types.FunctionType)
Help on class function in module builtins:

class function(object)
 |  function(code, globals, name=None, argdefs=None, closure=None)
 |
 |  Create a function object.
 |
 |  code
 |    a code object
 |  globals
 |    the globals dictionary
 |  name
 |    a string that overrides the name from the code object
 |  argdefs
 |    a tuple that specifies the default argument values
 |  closure
 |    a tuple that supplies the bindings for free variables
 ```

We can get a code object from an existing function

In [ ]:
print(lamb.__code__)

In [ ]:
import types
lambish = types.FunctionType(lamb.__code__, globals())
print(lambish(1))

or we can create one with `compile`

In [ ]:
code = compile('print(4)', 'no-file', 'exec')
compiled_fn = types.FunctionType(code, globals())
print(compiled_fn())

But how did `lambish` know about its argument?

In [ ]:
print(dir(lamb.__code__))

The documentation for the `inspect` module tells us that `co_argcount` is the number of arguments, and `co_varnames` is a tuple of the names of the arguments and then the names of the local variables

In [ ]:
print(lamb.__code__.co_argcount)
print(lamb.__code__.co_varnames)

But that't not mutable

In [ ]:
lamb.__code__.co_varnames = ('x', 'y')

Let's try looking at the "inside" of lambish

In [ ]:
import dis
dis.dis(lambish)

In [ ]:
dis.dis(lamb.__code__)

However, there is `signature` (which needs a callable as its argument)

In [ ]:
print(inspect.signature(lambish))

and that actually returns an instance of class `Signature`

In [ ]:
print(type(inspect.signature(lambish)))

In [ ]:
import dis
dis.dis(fn)

In [ ]:
dis.dis(fn.__code__)

In [ ]:
dis.dis(bare_f)

In [ ]:
print(dir(fn))

In [ ]:
print(fn.__dict__)

In [ ]:
print(fn.__call__)

In [ ]:
print(fn.__call__())

In [ ]:
print(dir(fn.f))

In [ ]:
save = fn.__code__
fn.__code__ = bare_f.__code__
print(fn(None, 'fred'))
fn.__code__ = save
print(fn())

-----------------

In [ ]:
a = lambda s, x: f'Lambda over #{s} and #{x}'

In [ ]:
print(a(1,2))

In [ ]:
dis.dis(a)

In [ ]:
def lf(s,x):
    pass

lf.__code__ = a.__code__

print(lf(1,2))

In [ ]:
lt = type('Function?', (), {})
lt.__code__ = a.__code__

In [ ]:
lt(1,2)

In [ ]:
print(dir(lf))

In [ ]:
import inspect


In [ ]:
print(inspect.isfunction(fn))
print(inspect.isfunction(fn.f))
print(inspect.isfunction(x.not_a_method))
print()
print(inspect.ismethod(fn))
print(inspect.ismethod(x.not_a_method))

In [ ]:
print(inspect.isclass(Example))
print(inspect.isclass(eo_class))

There's no point in asking if something is an instance, because everything is...

In [ ]:
inspect.signature(not_a_method)

In [ ]:
print(inspect.signature(bare_f))
print(inspect.getsource(bare_f))